In [ ]:
datasets = ["cat_dog_100"]

for dataset in datasets:
    print("dataset: ", dataset)

    # Cargamos los datos de entrenamiento y prueba desde los ficheros
    X_train = np.loadtxt("datasets/"+dataset+"_X_train.txt", dtype=np.uint8)
    y_train = np.loadtxt("datasets/"+dataset+"_y_train.txt", dtype=str)
    X_test = np.loadtxt("datasets/"+dataset+"_X_test.txt", dtype=np.uint8)
    y_test = np.loadtxt("datasets/"+dataset+"_y_test.txt", dtype=str)

    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))

In [ ]:
# Cargamos la imagen
N = 16
R = 3
# Convertimos la imagen a escala de grises
image = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)

result = feature_extraction_1(image, N, R)

print(len(result))

In [ ]:
def feature_extraction_1(image, N, R):
    # Obtenemos las dimensiones de la imagen
    height, width = image.shape[:2]

    #print("IMAGEN NxM: ",height, "x",width )

    # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N

    # Dividimos la imagen en bloques de N filas
    blocks = np.vsplit(image, num_blocks_rows)
   
    histograms = np.zeros((num_blocks_rows*num_blocks_cols, 256,1))

    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques de N columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, celda in enumerate(cells):
            
            vecinos = im_to_col(celda, R, R).T
            # Inicializamos el array de resultados para todos los vecinos
            result = np.zeros(vecinos.shape[0], np.uint8)
            # Iteramos sobre cada vecino
            for j, v in enumerate(vecinos):  
                puntoMedio = (R*R)//2;              
                # Obtenemos el valor del píxel central
                center_pixel = v[puntoMedio]
                
                # Inicializamos el valor binario a 0
                binary = ''
                # Iteramos sobre los vecinos de la celda, excluyendo el píxel central
                for x in range(len(v)):                    
                    if x == puntoMedio :
                        continue
                    if v[x] < center_pixel:
                        binary += '0'
                    else:
                        binary += '1'
                # Modificamos el valor del píxel central por el del número binario obtenido
                result[j] = int(binary, 2)

            # Calculamos el histograma del bloque
            histograms[j] = cv2.calcHist([result], [0], None, [256], [0, 256])

    return histograms.flatten()